In [1]:
from nanomesh.generator import Generator
import SimpleITK as sitk
from nanomesh.utils import show_slice, show_volume, generate_mesh_from_binary_image
import numpy as np
import math

In [10]:
import pygalmesh
class Pore3D(pygalmesh.DomainBase):
    def __init__(self):
        super().__init__()
        
        self.gen = Generator(680, math.sqrt(2)*680, 0.24*680)

        # Possible rotation/transformation of the coordinate system
        theta = math.pi * 1/180
        c = math.cos(theta)
        s = math.sin(theta)
        trans = np.array([
            [ c, 0, s],
            [ 0, 1, 0],
            [-s, 0, c]
        ])
        
        self.size = [100]*3
        self.res = [10]*3
        self.data = self.gen.generate_vect(self.size, self.res, transform=trans, bin_val=[0.,1.])
        self.data = self.data[:,:68,:]
        
        # idx = list(range(25,75))
        # semi_idx = list(range(25,100))
        # all_idx = list(range(self.size[0]))
        # flt = np.zeros((100,100,100))
        # flt[np.ix_(idx,idx,all_idx)] = 1.
        # flt[np.ix_(idx,all_idx,idx)] = 1.
        # flt[np.ix_(all_idx,idx,idx)] = 1.
        
        # flt[np.ix_(all_idx,range(75,100),all_idx)] = 1.
        
        
        # self.data = flt
        
        self.x = np.linspace(0,1.,self.data.shape[0])
        self.y = np.linspace(0,0.68,self.data.shape[1])
        self.z = np.linspace(0,1.,self.data.shape[2])
        
    def eval(self, x):
        ix = np.argmin(np.abs(x[0]-self.x))
        iy = np.argmin(np.abs(x[1]-self.y))
        iz = np.argmin(np.abs(x[2]-self.z))
        if self.data[ix,iy,iz] == 1:
            out = -1.
        else:
            out = 1.
        return out
    

In [11]:
pore = Pore3D()
im = sitk.GetImageFromArray(pore.data.astype('uint8'))
show_slice(im, dim='z',scale=2)

interactive(children=(IntSlider(value=49, description='z', max=99), Output()), _dom_classes=('widget-interact'…

In [12]:
import pygalmesh
mesh = generate_mesh_from_binary_image(im, h=[2.5]*3, perturb=False, 
                                       lloyd=False,
                                       odt=False,
                                       max_radius_surface_delaunay_ball=0.0,
                                       max_cell_circumradius=0.,
                                       max_facet_distance=1.,
                                       exude=True,
                                       min_facet_angle=0.,
                                       max_edge_size_at_feature_edges=1.5,
                                       max_circumradius_edge_ratio=0.,
                                       bounding_box=True)
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [14]:
mesh = pygalmesh.generate_periodic_mesh(
    Pore3D(),
    [0,0,0,1.,0.68,1.],
    max_cell_circumradius=0.05,
    min_facet_angle=30,
    max_radius_surface_delaunay_ball=0.05,
    max_facet_distance=0.025,
    max_circumradius_edge_ratio=2.0,
    number_of_copies_in_output=1,
    make_periodic=True,
    # odt=True,
    # lloyd=True,
    verbose=False,
    )

In [15]:
import pyvista as pv
pv.plot_itk(mesh)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…